## Import some packages

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import date
from datetime import datetime
from random import randrange
import glob
import os

## Start web browser using Selenium

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\menyh\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-2-6baf748b4601>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


Test that it works:

In [3]:
starting_link_to_scrape = "https://ingatlan.com/lista/elado+lakas+budapest?page=1"
driver.get(starting_link_to_scrape)
time.sleep(1)
base_source = driver.page_source
base_soup = BeautifulSoup(base_source, 'html.parser')

In [4]:
#base_soup

# Start

We want to scrapte www.indatlan.com which is the leading Hungarian housing website where you can find: apartments and houses to buy and rent. First we will focus on the buying market. On the buying market we need to handle apartments and houses differently because they have different properties and requires different scraping methods.

# SCRAPTING APARTMENTS

## Write a function that gets all the links from one page for apartments

In [5]:
def get_links_and_other_from_one_page(link = 'https://ingatlan.com/lista/elado+lakas+budapest?page=1',
                                     sleep_time = 1):
    
    # Navigate the web browser using Selenium to the web-page.
    starting_link_to_scrape = link
    driver.get(starting_link_to_scrape)
    time.sleep(sleep_time) #set sleep time
    base_source = driver.page_source
    base_soup = BeautifulSoup(base_source, 'html.parser')
    
    properties = base_soup.findAll('a', {'class': 'listing__link js-listing-active-area'})
    
    links = []
    prices = []
    adresses = []
    area_sizes = []
    room_numbers = []
    balcony_sizes = []

    for one_prop in properties:
        
        try:
            links.append(one_prop['href'])
        except:
            links.appen('no_link')
        
        try:
            price = one_prop.find('div', {'class': 'price'})
            prices.append(price.text)
        except:
            prices.append('NA')
        
        try:
            adress = one_prop.find('div', {'class': 'listing__address'})
            adresses.append(adress.text)
        except:
            adresses.append('NA')
            
        try:
            area_size = one_prop.find('div', {'class': 'listing__parameter listing__data--area-size'})
            area_sizes.append(area_size.text)
        except:
            area_sizes.append('NA')
            
        try:
            room_number = one_prop.find('div', {'class': 'listing__parameter listing__data--room-count'})
            room_numbers.append(room_number.text)
        except:
            room_numbers.append('NA')

        try:
            balcony_size = one_prop.find('div', {'class': 'listing__parameter listing__data--balcony-size'})
            balcony_sizes.append(balcony_size.text)
        except:
            balcony_sizes.append('NA')
            
    pandas_frame = {'links' : links,
                    'prices' : prices,
                    'adresses' : adresses,
                    'area_sizes' : area_sizes,
                    'room_numbers' : room_numbers,
                    'balcony_sizes' : balcony_sizes}
    
    df = pd.DataFrame(pandas_frame)
    
    return(df)

Test the function for one page:

In [6]:
links_page1 = get_links_and_other_from_one_page('https://ingatlan.com/lista/elado+lakas+budapest?page=4')

In [7]:
links_page1

,links,prices,adresses,area_sizes,room_numbers,balcony_sizes
0,/vii-ker/elado+lakas/tegla-epitesu-lakas/31962796,20.9 M Ft,"Péterfy Sándor utca, VII. kerület",44 m² terület,2 szoba,NA
1,/xi-ker/elado+lakas/tegla-epitesu-lakas/32630050,55 M Ft,"Bocskai út 34a, XI. kerület",53 m² terület,2 szoba,NA
2,/xiii-ker/elado+lakas/tegla-epitesu-lakas/3260...,70.04 M Ft,"Cserhalom utca, XIII. kerület",69 m² terület,3 szoba,10 m² erkély
3,/vii-ker/elado+lakas/tegla-epitesu-lakas/32558707,36.9 M Ft,"Cserhát utca, VII. kerület",71 m² terület,2 szoba,NA
4,/xiii-ker/elado+lakas/tegla-epitesu-lakas/3232...,64.49 M Ft,"Tahi utca 59-61, XIII. kerület",57 m² terület,3 szoba,15 m² erkély
5,/xiii-ker/elado+lakas/tegla-epitesu-lakas/2921...,72 M Ft,"Angyalföld, XIII. kerület",77 m² terület,3 szoba,10.4 m² erkély
6,/viii-ker/elado+lakas/tegla-epitesu-lakas/3261...,49.9 M Ft,"Rákóczi út, VIII. kerület",71 m² terület,2 szoba,2 m² erkély
7,/xi-ker/elado+lakas/tegla-epitesu-lakas/32576925,99.99 M Ft,"Kosztolányi Dezső tér, XI. kerület",155 m² terület,5 szoba,25 m² erkély
8,/viii-ker/elado+lakas/tegla-epitesu-lakas/3227...,49.7 M Ft,"Fiumei út 5, VIII. kerület",40 m² terület,2 szoba,NA
9,/xi-ker/elado+lakas/tegla-epitesu-lakas/31807833,86.88 M Ft,"Budafoki út 215, XI. kerület",72 m² terület,3 szoba,37.24 m² erkély


## Write a function that scrapes all pages using the previus functions

In [8]:
def get_properties_for_many_pages(web_page_base = 'https://ingatlan.com/lista/elado+lakas+budapest?page=',
                                  from_page = 1, to_page = 10,
                                  save_csv = True):
    
    print('Started Scraping')
    print('Scraping the Following Page:')
    
    all_data = pd.DataFrame()
    web_page_base = 'https://ingatlan.com/lista/elado+lakas+budapest?page='
    
    for i in range(from_page, to_page):
        try:
            web_page_link = web_page_base + str(i)
            print(web_page_link)
            one_data = get_links_and_other_from_one_page(web_page_link)            
            all_data = pd.concat([all_data,one_data], axis=0, ignore_index=True)
            
        except:
            continue
    
    all_data['webpage_all'] = 'https://ingatlan.com' + all_data['links']
    
    if save_csv:
        now = datetime.now()
        dt_string = now.strftime("%Y%m%d_%H%M%S")
        file_name_for_saving = str('base_data' + dt_string + '.csv')
        
        all_data.to_csv(file_name_for_saving, encoding='utf-8', index=False)
    
    return(all_data)

In [9]:
#list_of_aps = get_properties_for_many_pages(from_page=1,
#                                            to_page=1600,
#                                            save_csv = True)

## Import the file that we want to use to be the basis of further scraping = we want all the apartaments that we have just scaped

In [10]:
filename_of_apt_file = 'base_data20211221_035607.csv'
list_of_aps_data = pd.read_csv(filename_of_apt_file)
list_of_aps_data

,links,prices,adresses,area_sizes,room_numbers,balcony_sizes,webpage_all
0,/xi-ker/elado+lakas/tegla-epitesu-lakas/31973403,53.75 M Ft,"Budafoki út 215, XI. kerület",41 m² terület,2 szoba,12.93 m² erkély,https://ingatlan.com/xi-ker/elado+lakas/tegla-...
1,/xi-ker/elado+lakas/tegla-epitesu-lakas/30457153,102.9 M Ft,"Rétköz utca, XI. kerület",94 m² terület,4 szoba,23.42 m² erkély,https://ingatlan.com/xi-ker/elado+lakas/tegla-...
2,/xi-ker/elado+lakas/tegla-epitesu-lakas/32605162,52.06 M Ft,"Szerenád utca 4, XI. kerület",48 m² terület,3 szoba,8.57 m² erkély,https://ingatlan.com/xi-ker/elado+lakas/tegla-...
3,/xiii-ker/elado+lakas/panel-lakas/32632610,56 M Ft,"Kárpát utca, XIII. kerület",68 m² terület,3 szoba,4 m² erkély,https://ingatlan.com/xiii-ker/elado+lakas/pane...
4,/xiii-ker/elado+lakas/tegla-epitesu-lakas/3187...,56.49 M Ft,"Lehel utca 9, XIII. kerület",53 m² terület,2 szoba,8 m² erkély,https://ingatlan.com/xiii-ker/elado+lakas/tegl...
...,...,...,...,...,...,...,...
30728,/v-ker/elado+lakas/tegla-epitesu-lakas/6248324,540 M Ft,"Vörösmarty tér, V. kerület",260 m² terület,3 szoba,NaN,https://ingatlan.com/v-ker/elado+lakas/tegla-e...
30729,/v-ker/elado+lakas/tegla-epitesu-lakas/6181917,349 M Ft,"Belgrád rakpart, V. kerület",246 m² terület,5 + 1 fél szoba,NaN,https://ingatlan.com/v-ker/elado+lakas/tegla-e...
30730,/vii-ker/elado+lakas/tegla-epitesu-lakas/5196070,100.04 M Ft,"Damjanich utca, VII. kerület",109 m² terület,3 szoba,6 m² erkély,https://ingatlan.com/vii-ker/elado+lakas/tegla...
30731,/viii-ker/elado+lakas/tegla-epitesu-lakas/4563923,79.8 M Ft,"Népszínház utca, VIII. kerület",102 m² terület,3 + 1 fél szoba,3 m² erkély,https://ingatlan.com/viii-ker/elado+lakas/tegl...


### Get one random apartment's link

In [11]:
random_number = randrange(len(list_of_aps_data))
print(random_number)
one_link = list_of_aps_data['webpage_all'][random_number]
print(one_link)

2040
https://ingatlan.com/i-ker/elado+lakas/tegla-epitesu-lakas/32630625


## Write a function that can srcrape one apartment

By using this function we can get the data for one apartment in a Pandas data frame

In [12]:
def scrape_one_apartment(link = 'https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/32076067',
                        sleep_time = 1):
    
    starting_link_to_scrape = link
    driver.get(starting_link_to_scrape)
    time.sleep(sleep_time)
    base_source = driver.page_source
    time.sleep(sleep_time) #sleep time
    base_soup = BeautifulSoup(base_source, 'html.parser')
    
    try:
        parameters_base = base_soup.find('dl', {'class': 'parameters'})
        #print(parameters_base)
        parameterNames_raw = parameters_base.findAll('dt', {'class': 'parameterName'})
        parameterValues__raw = parameters_base.findAll('dd', {'class': 'parameterValue'})
        
        parameterNames = []
        parameterValues = []
        
        for parameterName in parameterNames_raw:
            parameterName_text = parameterName.text
            parameterName_text = re.sub(r'[\W_]+', '', parameterName_text)
            parameterNames.append(parameterName_text)
            
        for parameterValue in parameterValues__raw:
            parameterValue_text = parameterValue.text
            parameterValue_text = re.sub(r'[\W_]+', '', parameterValue_text)
            parameterValues.append(parameterValue_text)
    
        dataframe = pd.DataFrame(data = [parameterValues], columns = [parameterNames]) 
        dataframe['link'] = link
        dataframe['error'] = 'no'
    
    except:
        dataframe = pd.DataFrame(data = ['yes'], columns = ['error'])
    
    return(dataframe)

Test the function

In [13]:
one_ap_params = scrape_one_apartment(link=one_link)

In [14]:
one_ap_params

,Ingatlanállapota,Építéséve,Komfort,Energiatanúsítvány,Emelet,Épületszintjei,Lift,Belmagasság,Fűtés,Légkondicionáló,...,Akadálymentesített,FürdőésWC,Tájolás,Kilátás,Erkélymérete,Kertkapcsolatos,Tetőtér,Parkolás,link,error
0,felújított,1950előtt,összkomfortos,nincsmegadva,1,2,nincsmegadva,3mnélmagasabb,elektromos,nincsmegadva,...,igen,nincsmegadva,délnyugat,utcai,12m²,nem,nincsmegadva,utcaközterületingyenes,https://ingatlan.com/i-ker/elado+lakas/tegla-e...,no


## As of now we have all the components to scrape the data for all the apartments

We need to create all the links that are refering to the apartments (we have them it our pandas data frame) and loop through them

In [15]:
all_links_for_apartments_all = list_of_aps_data['webpage_all']
print(len(all_links_for_apartments_all))
all_links_for_apartments = list(dict.fromkeys(all_links_for_apartments_all))
print(len(all_links_for_apartments))

30733
30733


We have this many apartment as above

## Write a function what loops throuh all the apartments and saves them

In [16]:
def get_all_apt_params(links = all_links_for_apartments[1:5], save_csv = True, filename_input = 'apt_data'):
    
    all_data = pd.DataFrame()
    i = 0
    all_links_len = len([links])
    time1 = datetime.now()
    
    for link in [links]:
        
        now = datetime.now()
        dt_string = now.strftime("%Y.%m.%d %H:%M:%S")
        
        try:
            print(str(i) + ') ' + str(link))
            one_data = scrape_one_apartment(link)
            one_data['download_time'] = dt_string
            
            all_data = pd.concat([all_data,one_data], axis=0, ignore_index=True)
            i = i + 1
            
            #print(str('left: ') + str(all_links_len - i + 1))
            
            divider = 25
            if i % divider == 0:
                
                time2 = datetime.now()
                
                time_left = str(round((all_links_len - i + 1) / divider * (time2 - time1).total_seconds() / 60))
                #print("approx mins left: ", time_left)
                
                time1 = datetime.now()
                
        except:
            i = i + 1
            continue
    
    if save_csv:
        
        now = datetime.now()
        dt_string = now.strftime("%Y%m%d_%H%M%S")
        apt_data_filename = str(filename_input) + str(dt_string) + '.csv'
        
        all_data.to_csv(apt_data_filename, encoding='utf-8', index=False)
    
    return(all_data)

In [17]:
apt_data = get_all_apt_params(links = all_links_for_apartments[0], save_csv = True)

0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/31973403


## Run this code in chunk to be able to figure out if there is an error

In [18]:
folder_name = 'all_apartments_211221'
filname_input_string = str(folder_name) + '/apt_data'
filname_input_string

'all_apartments_211221/apt_data'

In [19]:
a = 15000

while a <= len(all_links_for_apartments):
    try:
        get_all_apt_params(links = all_links_for_apartments[a], save_csv = True, filename_input=filname_input_string)
        a = a + 1
        print(str(a) + '/' + str(len(all_links_for_apartments)))
    except:
        a = a + 1
        continue

0) https://ingatlan.com/ii-ker/elado+lakas/csuszozsalus-lakas/32615078
15001/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-lakas/32592680
15002/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/32405640
15003/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/32011839
15004/30733
0) https://ingatlan.com/viii-ker/elado+lakas/panel-lakas/32622458
15005/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/31495901
15006/30733
0) https://ingatlan.com/iii-ker/elado+lakas/tegla-epitesu-lakas/32471888
15007/30733
0) https://ingatlan.com/xviii-ker/elado+lakas/tegla-epitesu-lakas/32457184
15008/30733
0) https://ingatlan.com/xx-ker/elado+lakas/tegla-epitesu-lakas/32417267
15009/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/32356013
15010/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/21660649
15011/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/32561294
1501

15099/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/32390944
15100/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/32369387
15101/30733
0) https://ingatlan.com/i-ker/elado+lakas/tegla-epitesu-lakas/32349143
15102/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/32630186
15103/30733
0) https://ingatlan.com/xvii-ker/elado+lakas/panel-lakas/32607698
15104/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/32492755
15105/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/32456893
15106/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/32456850
15107/30733
0) https://ingatlan.com/xviii-ker/elado+lakas/panel-lakas/32601224
15108/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/32389325
15109/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/31937693
15110/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-lakas/32605386

15197/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-lakas/32588880
15198/30733
0) https://ingatlan.com/xvi-ker/elado+lakas/tegla-epitesu-lakas/32479793
15199/30733
0) https://ingatlan.com/xx-ker/elado+lakas/panel-lakas/32546580
15200/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/panel-lakas/32382743
15201/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/30021113
15202/30733
0) https://ingatlan.com/xix-ker/elado+lakas/panel-lakas/32578557
15203/30733
0) https://ingatlan.com/xix-ker/elado+lakas/tegla-epitesu-lakas/32483364
15204/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/32630896
15205/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/32468008
15206/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/31405879
15207/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/32331929
15208/30733
0) https://ingatlan.com/xviii-ker/elado+lakas/tegla-epitesu-lakas/31785996
15209

15295/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/28879250
15296/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/32586470
15297/30733
0) https://ingatlan.com/x-ker/elado+lakas/tegla-epitesu-lakas/32586229
15298/30733
0) https://ingatlan.com/iv-ker/elado+lakas/panel-lakas/31695847
15299/30733
0) https://ingatlan.com/xvi-ker/elado+lakas/tegla-epitesu-lakas/31459999
15300/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/32602684
15301/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/panel-lakas/32600578
15302/30733
0) https://ingatlan.com/x-ker/elado+lakas/panel-lakas/32514415
15303/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/32487952
15304/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/32485017
15305/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/31784411
15306/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/32605251
15307/30

15394/30733
0) https://ingatlan.com/xii-ker/elado+lakas/tegla-epitesu-lakas/32605732
15395/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/32629734
15396/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/32624156
15397/30733
0) https://ingatlan.com/xvi-ker/elado+lakas/tegla-epitesu-lakas/32595626
15398/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/32538476
15399/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/31998795
15400/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/32630000
15401/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/32470515
15402/30733
0) https://ingatlan.com/iv-ker/elado+lakas/tegla-epitesu-lakas/32404442
15403/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/32135923
15404/30733
0) https://ingatlan.com/xvii-ker/elado+lakas/csuszozsalus-lakas/32043029
15405/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu

...

30276/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/30256376
30277/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/30256373
30278/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/30256367
30279/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/30256364
30280/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/30256304
30281/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-lakas/30250820
30282/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/30250580
30283/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/30247169
30284/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/30241844
30285/30733
0) https://ingatlan.com/xii-ker/elado+lakas/tegla-epitesu-lakas/30234617
30286/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/30230489
30287/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-

30374/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/29819357
30375/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/29818283
30376/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/29816660
30377/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/29814965
30378/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/29814875
30379/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/29814797
30380/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/29809460
30381/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/29808041
30382/30733
0) https://ingatlan.com/xviii-ker/elado+lakas/tegla-epitesu-lakas/29803328
30383/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-lakas/29803043
30384/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/29801861
30385/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lak

30472/30733
0) https://ingatlan.com/ii-ker/elado+lakas/tegla-epitesu-lakas/29326553
30473/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/29326232
30474/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/29323664
30475/30733
0) https://ingatlan.com/xii-ker/elado+lakas/tegla-epitesu-lakas/29313788
30476/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/29298059
30477/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/29295743
30478/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/29288780
30479/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/29287877
30480/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/29282042
30481/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/29269256
30482/30733
0) https://ingatlan.com/xiv-ker/elado+lakas/tegla-epitesu-lakas/29262461
30483/30733
0) https://ingatlan.com/xv-ker/elado+lakas/tegla-epitesu-lakas

30570/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/28033259
30571/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/27941717
30572/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/27924158
30573/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/27910355
30574/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/27889061
30575/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/27887828
30576/30733
0) https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/27879251
30577/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/27871847
30578/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/27855719
30579/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-epitesu-lakas/27840083
30580/30733
0) https://ingatlan.com/viii-ker/elado+lakas/tegla-epitesu-lakas/27824504
30581/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lak

30668/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/23385683
30669/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/23385678
30670/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/23385170
30671/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/23385098
30672/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/23370083
30673/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/23353914
30674/30733
0) https://ingatlan.com/ix-ker/elado+lakas/tegla-epitesu-lakas/23115605
30675/30733
0) https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/23029328
30676/30733
0) https://ingatlan.com/xiii-ker/elado+lakas/tegla-epitesu-lakas/22999111
30677/30733
0) https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/22974231
30678/30733
0) https://ingatlan.com/iii-ker/elado+lakas/tegla-epitesu-lakas/22944807
30679/30733
0) https://ingatlan.com/vii-ker/elado+lakas/tegla-e

## Read in apartment data

For this we need to read in all the files in the previous folder

In [20]:
# setting the path for joining multiple files
files = os.path.join(folder_name, '*.csv')
# list of merged files returned
files_to_join = glob.glob(files)

In [21]:
len(files_to_join)

31225

In [22]:
all_apt = pd.concat(pd.read_csv, files_to_join), ignore_index=True)

SyntaxError: unmatched ')' (<ipython-input-22-8703e4ee4527>, line 1)

In [ ]:
all_apt